In [1]:
import numpy as np
import pandas as pd
import os

new_dir = r"C:\Users\Sanaiyah\Desktop\Yolo-research\automobile\VIT-yolo\runs"
os.chdir(new_dir)
print("Current Directory:", os.getcwd())

Current Directory: C:\Users\Sanaiyah\Desktop\Yolo-research\automobile\VIT-yolo\runs


In [3]:
!pip install --upgrade ultralytics timm torch


  Using cached torch-2.6.0-cp312-cp312-win_amd64.whl.metadata (28 kB)


In [4]:
# hybrid_mobilevit_yolo.py
import torch
from torch import nn
import timm
from ultralytics import YOLO

# 1) Multi-scale MobileViT extractor
class MobileViTFeatureExtractor(nn.Module):
    def __init__(self, model_name='mobilevit_s'):
        super().__init__()
        # timm features_only returns a list of stage feature maps
        self.backbone = timm.create_model(model_name, pretrained=True, features_only=True)
        # last three stages
        chs = [self.backbone.feature_info[i]['num_chs'] for i in (-3, -2, -1)]
        self.proj = nn.ModuleList([nn.Conv2d(c, 256, 1) for c in chs])

    def forward(self, x):
        feats = self.backbone(x)            # list of feature maps
        sel   = [feats[i] for i in (-3, -2, -1)]
        return tuple(p(f) for p, f in zip(self.proj, sel))

# 2) Hybrid wrapper replacing YOLO backbone
class MobileViTYOLO(nn.Module):
    def __init__(self, num_classes=8):
        super().__init__()
        self.backbone = MobileViTFeatureExtractor()

        # load pretrained YOLOv8n weights
        self.yolo = YOLO('yolov8n.pt')
        # override to KITTI's 8 classes
        names = ["car","pedestrian","van","cyclist","truck","misc","tram","person_sitting"]
        self.yolo.model.nc = num_classes
        self.yolo.model.names = names
        self.yolo.model.yaml['nc'] = num_classes
        self.yolo.model.yaml['names'] = names

        # replace original backbone (first modules) with MobileViT extractor
        modules = list(self.yolo.model.model)
        backbone_cut = 10  # adjust if your YOLO version differs
        head = modules[backbone_cut:]
        self.yolo.model.model = nn.Sequential(self.backbone, *head)

    def forward(self, x):
        return self.yolo(x)

# 3) Training entrypoint with workers parameter
def train(
    data_yaml: str = "dataset.yaml",
    epochs: int = 100,
    imgsz: int = 640,
    batch_size: int = 8,
    lr: float = 1e-4,
    workers: int = 0
):
    model = MobileViTYOLO(num_classes=8)
    # YOLOv8 train API
    model.yolo.train(
        data=data_yaml,
        epochs=epochs,
        imgsz=imgsz,
        rect=True,
        batch=batch_size,
        workers=workers,
        optimizer='AdamW',
        lr0=lr
    )

if __name__ == "__main__":
    train(
        data_yaml="C:/Users/Sanaiyah/Desktop/Yolo-research/automobile/VIT-yolo/dataset.yaml",
        epochs=200,
        imgsz=640,
        batch_size=8,
        lr=1e-4,
        workers=0
    )


New https://pypi.org/project/ultralytics/8.3.111 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.110  Python-3.12.3 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 6GB Laptop GPU, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:/Users/Sanaiyah/Desktop/Yolo-research/automobile/VIT-yolo/dataset.yaml, epochs=200, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=0, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=True, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, 

train: Scanning C:\Users\Sanaiyah\Desktop\Yolo-research\automobile\VIT-yolo\labels\train.cache... 7481 images, 0 backgrounds, 0 corrupt: 100%|██████████| 7481/7481 [00:00<?, ?it/s]

WARNING  'rect=True' is incompatible with DataLoader shuffle, setting shuffle=False


val: Fast image access  (ping: 0.10.0 ms, read: 470.5102.3 MB/s, size: 790.1 KB)


val: Scanning C:\Users\Sanaiyah\Desktop\Yolo-research\automobile\VIT-yolo\labels\train.cache... 7481 images, 0 backgrounds, 0 corrupt: 100%|██████████| 7481/7481 [00:00<?, ?it/s]


...

                   all       7481      40570      0.764      0.649      0.722      0.467

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    200/200     0.547G      0.977     0.6412     0.9893         10        640: 100%|██████████| 936/936 [05:21<00:00,  2.91it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 468/468 [03:50<00:00,  2.03it/s]


                   all       7481      40570      0.764      0.649      0.722      0.467

200 epochs completed in 31.460 hours.
Optimizer stripped from runs\detect\train6\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train6\weights\best.pt, 6.2MB

Validating runs\detect\train6\weights\best.pt...
Ultralytics 8.3.110  Python-3.12.3 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 6GB Laptop GPU, 6144MiB)
Model summary (fused): 72 layers, 3,007,208 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 468/468 [03:51<00:00,  2.02it/s]


                   all       7481      40570      0.764      0.649      0.722      0.467
                   car       6684      28742      0.889      0.846      0.923      0.694
            pedestrian       1779       4487      0.811      0.475      0.617      0.323
                   van       2145       2914      0.833      0.749       0.84      0.598
               cyclist       1141       1627      0.674      0.595      0.616      0.332
                 truck       1036       1094      0.889      0.863      0.932      0.696
                  misc        778        973      0.782      0.501      0.635      0.372
                  tram        349        511      0.727      0.835      0.874      0.583
        person_sitting         99        222      0.505      0.329       0.34      0.142
Speed: 0.3ms preprocess, 3.7ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs\detect\train6


In [ ]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import seaborn as sns

# ✅ Load your trained model
model = YOLO(r"C:\Users\Sanaiyah\Desktop\Yolo-research\automobile\VIT-yolo\runs\runs\detect\train9\weights\best.pt")

# ✅ Run validation (uses val split from dataset.yaml)
metrics = model.val()

# ✅ Extract final results
precision = metrics.box.mp
recall = metrics.box.mr
map50 = metrics.box.map50
map5095 = metrics.box.map
f1_score = 2 * precision * recall / (precision + recall + 1e-6)

# ✅ Print metrics
print("\n✅ FINAL RESULTS:")
print(f"Precision:     {precision:.4f}")
print(f"Recall:        {recall:.4f}")
print(f"F1-Score:      {f1_score:.4f}")
print(f"mAP@0.5:       {map50:.4f}")
print(f"mAP@0.5:0.95:  {map5095:.4f}")



Ultralytics 8.3.110  Python-3.12.3 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 6GB Laptop GPU, 6144MiB)
Model summary (fused): 72 layers, 3,007,208 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 275.778.5 MB/s, size: 763.6 KB)


val: Scanning C:\Users\Sanaiyah\Desktop\Yolo-research\automobile\VIT-yolo\labels\train.cache... 7481 images, 0 backgrounds, 0 corrupt: 100%|██████████| 7481/7481 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 468/468 [03:33<00:00,  2.19it/s]


                   all       7481      40570      0.689      0.581      0.637      0.391
                   car       6684      28742      0.827      0.835      0.899      0.648
            pedestrian       1779       4487      0.766      0.428      0.542      0.268
                   van       2145       2914      0.779      0.638      0.748      0.501
               cyclist       1141       1627      0.593      0.463      0.505      0.264
                 truck       1036       1094      0.841      0.777       0.86      0.614
                  misc        778        973      0.774      0.377      0.507      0.289
                  tram        349        511      0.576      0.775      0.773      0.455
        person_sitting         99        222      0.361      0.356      0.258     0.0927
